In [73]:
import sys
import io
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler,LabelEncoder
from sklearn.decomposition import PCA
import sqlite3
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
import datetime as dt
from sklearn import linear_model
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
import json
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import openai
import pandas as pd
import sqlite3
from openai.api_resources.completion import Completion
import sqlparse
import time
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Token

In [3]:
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
sql = "select * from fires"

df = pd.read_sql_query(sql, cnx)
df.head()
str(df.columns)

"Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape'], dtype='object')"

In [4]:
df.head()


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,None,None,None,None,None,2005,2453403.5,33,1300,9.0,Miscellaneous,2453403.5,33.0,1730,0.10,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,13,AAC0,PIGEON,None,None,None,None,None,2004,2453137.5,133,0845,1.0,Lightning,2453137.5,133.0,1530,0.25,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,021,A32W,SLACK,None,None,None,None,None,2004,2453156.5,152,1921,5.0,Debris Burning,2453156.5,152.0,2024,0.10,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,6,None,DEER,None,None,None,None,None,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,1400,0.10,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,7,None,STEVENOT,None,None,None,None,None,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,1200,0.10,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [5]:
f = open("columns.txt", "r")

col_txt = f.read()

In [6]:
openai.api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'


In [84]:
class DataAnalysisAgent:
    def __init__(self, db_path):
        self.connection = sqlite3.connect(db_path)
        self.cursor = self.connection.cursor()
        self.valid_columns = self.fetch_column_names()
        self.valid_tables = {'fires'}

    def fetch_column_names(self):
        """ Fetches all column names from the 'fires' table to ensure query accuracy. """
        self.cursor.execute("PRAGMA table_info(fires)")
        return {item[1] for item in self.cursor.fetchall()}  # Set of column names


    def is_valid_sql_columns_table(self, sql_query):
        """ Check if the SQL is syntactically correct and if it contains valid columns and tables. """
        try:
            parsed = sqlparse.parse(sql_query)[0]
            tokens = [token for token in parsed.tokens if not token.is_whitespace]
            print(tokens)

            found_tables = set()
            found_columns = set()
            for token in tokens:
                if(1):
                    value = token.value
                    if value in self.valid_columns:
                        found_columns.add(value)
                    elif value in self.valid_tables:
                        found_tables.add(value)
            print(found_tables)
            print(found_columns)

            if not found_tables.issubset(self.valid_tables) or found_tables == set():
                print("Query contains invalid table names.")
                return False
            if not found_columns.issubset(self.valid_columns) :
                print("Query contains invalid column names.")
                return False
            return True

        except sqlparse.exceptions.SQLParseError:
            return False

    def is_valid_sql_columns(self, sql_query):
        """ Check if the SQL is syntactically correct and if it contains valid columns. """
        try:
            parsed = sqlparse.parse(sql_query)[0]
            tokens = {token.value for token in parsed.tokens if token.ttype is sqlparse.tokens.Name}
            if not tokens.issubset(self.valid_columns):
                print("Query contains invalid column names.")
                return False
            return True
        except sqlparse.exceptions.SQLParseError:
            return False

    def is_valid_sql(self,sql_query):
        try:
            sqlparse.parse(sql_query)
            return True
        except sqlparse.exceptions.SQLParseError:
            return False
        



    def generate_sql_query(self, user_query):
        """
        Uses OpenAI's GPT-3.5 Turbo to generate an SQL query based on the user's natural language query.
        
        Args:
            user_query (str): A natural language query from the user.

        Returns:
            str: SQL query string generated by the AI.
            dict: Any SQL parameters for safe query execution (not used here for simplicity).
        """
        prompt = ''' You are an assistant trained to convert natural language to SQL.
        <INST> 
            
        DATABASE DETAILS:
        - Database: SQLite
        - Table Name: fires
        - cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
        - USE cnx to query the database using sqlite commands


        MEMORY:
        Context: The 'fires' table contains a comprehensive spatial database of wildfires that occurred in the United States from 1992 to 2015. This data supports national Fire Program Analysis (FPA) systems. The records, sourced from federal, state, and local fire organizations, include essential elements such as discovery date, final fire size, and point locations precise to at least the Public Land Survey System (PLSS) section (1-square mile grid). The data conforms to standards set by the National Wildfire Coordinating Group (NWCG) and includes error-checking and redundancy removal. This data publication, known as the Fire Program Analysis fire-occurrence database (FPA FOD), comprises 1.88 million geo-referenced wildfire records, representing 140 million acres burned over a 24-year period.

        Column Information:

        '''
        prompt+= col_txt
        prompt+='''
        <\INST>
        Assistant: ```sql
        <SQL>
        ```
        '''
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Specify the appropriate model
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": user_query}
            ]
        )
        sql_query = response['choices'][0]['message']['content']
        pattern = r"```sql(.*?)```"
        print(sql_query)
        try:
            # Search for the pattern in the text
            match = re.search(pattern, sql_query, re.DOTALL)
            sql_extracted = match.group(1).strip()

            if match:
                # Extract the SQL query, stripping any leading or trailing whitespace
                if(self.is_valid_sql(sql_extracted)):
                    print("Valid SQL query")
                else:
                    print("Regenerate the query") #return to base
                if(self.is_valid_sql_columns(sql_extracted)):
                    print("Valid SQL query and columns")
                else:
                    print("Regenerate the query") #return to base
                return match.group(1).strip(),{}
            else:
                return "",{}

            return sql_query,{}
        except Exception as e:
            print(f"Failed to generate SQL query: {str(e)}")
            return "", {}

#     def execute_query(self, query):
#         return pd.read_sql_query(query, self.connection)
    def execute_query(self, query, timeout=10, max_rows = 1000000):  # Timeout in seconds for query execution
#         query = f"{query} LIMIT {max_rows}" 
        start_time = time.time()
        try:
            df = pd.read_sql_query(query, self.connection)
            if time.time() - start_time > timeout:
                raise TimeoutError("Query execution exceeded the time limit.")
            return df
        except TimeoutError as te:
            print(te)
            return pd.DataFrame()  # Return empty DataFrame on timeout
        except Exception as e:
            print(f"Error executing query: {str(e)}")
            return pd.DataFrame()

    def analyze_data(self, dataframe):
        # Perform analysis like aggregations, statistical tests, etc.
        return dataframe.describe()
    
    def get_query_plan(self,query):
        plan_query = f"EXPLAIN {query}"
        return pd.read_sql_query(plan_query, self.connection)


    def handle_query(self, user_query):
        sql_query, params = self.generate_sql_query(user_query)
        print("query")
        print(sql_query)
        try:
            data = self.execute_query(sql_query)
            print(data)
            if data.empty:
                return "No data returned."
            else:
                result = self.analyze_data(data)
                return result
        except Exception as e:
            return f"Error executing query: {str(e)}"


# Usage
agent = DataAnalysisAgent('FPA_FOD_20170508.sqlite')

# result = agent.handle_query("What is the average fire size?")
# print(result)


In [90]:
#TESTING

import unittest

class TestDataAnalysisAgent(unittest.TestCase):

    def setUp(self):
        self.agent = DataAnalysisAgent('FPA_FOD_20170508.sqlite')

    def test_fetch_column_names(self):
        columns = self.agent.fetch_column_names()
        self.assertIsInstance(columns, set)
        self.assertTrue('FIRE_SIZE' in columns)

    def test_is_valid_sql(self):
        valid_sql = "SELECT * FROM fires;"
        invalid_sql = "SELECT * FROM unknown_table;"
        self.assertTrue(self.agent.is_valid_sql_columns_table(valid_sql))
        self.assertFalse(self.agent.is_valid_sql_columns_table(invalid_sql))

    def test_sql_query_generation(self):
        response = self.agent.generate_sql_query("average fire size")
        self.assertNotEqual(response, ("", {}))  # Assuming non-empty returns on valid prompts

    def test_execute_query(self):
        query = "SELECT * FROM fires LIMIT 10;"
        result = self.agent.execute_query(query)
        self.assertFalse(result.empty)

    def test_query_handling(self):
        # Test the end-to-end process of handling a query
        result = self.agent.handle_query("What is the average fire size?")
        print("result")
        print(str(result))
        self.assertNotEqual(str(result), "")
        self.assertIn('mean', str(result))  # Assuming 'mean' would be part of the description output

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)



...

[<DML 'SELECT' at 0x287715960>, <Wildcard '*' at 0x2862D5AE0>, <Keyword 'FROM' at 0x2862D7820>, <Identifier 'fires' at 0x2863076D0>, <Punctuation ';' at 0x2862D6BC0>]
{'fires'}
set()
[<DML 'SELECT' at 0x2862D64A0>, <Wildcard '*' at 0x2862D5D80>, <Keyword 'FROM' at 0x2862D7700>, <Identifier 'unknow...' at 0x2863075D0>, <Punctuation ';' at 0x2862D5FC0>]
set()
set()
Query contains invalid table names.
```sql
SELECT AVG(FIRE_SIZE) AS Average_Fire_Size
FROM fires;
```
Valid SQL query
Valid SQL query and columns
query
SELECT AVG(FIRE_SIZE) AS Average_Fire_Size
FROM fires;


.

   Average_Fire_Size
0          74.520158
result
       Average_Fire_Size
count           1.000000
mean           74.520158
std                  NaN
min            74.520158
25%            74.520158
50%            74.520158
75%            74.520158
max            74.520158


.
----------------------------------------------------------------------
Ran 5 tests in 1.672s

OK


```sql
SELECT AVG(FIRE_SIZE)
FROM fires;
```
Valid SQL query
Valid SQL query and columns
